## Improvement from QA_v1:
### 1. Change the politeness model to the sentiment model, which is powered by Bert and works better

### 2. Test more cases for completeness

## Outline:
### 1. Input transcripts and grammars

### 2. Sentencizer transcripts

### 3. Mapping sentences to each category

### 4. Calculate sentiment score for each part

### To Do

#### 1. Solve bugs in aspect matching, and add some examples to each aspect
#### 2. Blur the boundary between different labeled classes (say the prob for classes are 75% and 80%, we can categorize it to both the classes)
#### 3. nlp_aspect_matching (bool_group = False) causes error, need debug
#### 4. grammars pos also have error, seperating sentences in weird way.

In [1]:
# Load packages
import nltk
import spacy
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load customized packages
from customized_pos import *
from pos_helper import *
from aspect_matching import *
from politeness_helper import *
from sentiment_helper import *

# Load spacy nlp model
nlp = spacy.load('en_core_web_lg')
# spacy version = 2.3.5 can use the following line of sentence
# nlp.add_pipe(pos_postprocessor_pipe, name="pos_postprocessor", after='tagger')
nlp.add_pipe("pos_postprocessor_pipe", after='tagger')

/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<function customized_pos.pos_postprocessor_pipe(doc)>

In [2]:
# Load sentiment model
tokenizer = AutoTokenizer.from_pretrained("Souvikcmsa/SentimentAnalysisDistillBERT")
sentiment_model = AutoModelForSequenceClassification.from_pretrained("Souvikcmsa/SentimentAnalysisDistillBERT")

### Section 1: Input transcripts and grammars

In [3]:
# Load from text file
with open ("data_collection/transcripts.txt") as f:
    transcripts_ls = f.readlines()

In [4]:
transcripts_ls

['ya hello good afternoon speak to nanny seah please afternoon miss nanny my name is jaguar shao and im actually calling from insurance company miss nanny good time to speak for a while this is regarding our partnership charter_plus members and as a result of that insurance company actually formulated a very special thirtieth anniversay insurance bundle right called the i thirty\n',
 'hello good afternoon just speak to miss leon michael from income ntuc free for one_or_two minutes if you are not busy okay calling behalf of your adviser xiao guo okay because we having this anniversary plan for the family i just check again you are single_or_married\n']

In [5]:
grammars = r""" 
    JP: {<JJ.*>}
    NP: {<JP|CD>*<PRP.*|DT|NN.*>+}
    PP: {<IN|TO|RP><NP|VB.*>} 
    VP: {<VB.*|RB.*>+<JP>*}
    VP: {<VB.*|RB.*>+<PP|NP>*}
    Sentence: {<UH>*<JP|NP>+<MD|IN>*<VP|PP|NP>+}
    Sentence: {<VP|PP|NP|JP>+}
    Question: {<MD|WDT|DP|WRB|><MD>*<Sentence|NP|PP|VP|JP>}
"""
# grammars = r""" 
#     NP: {<PRP.*|DT|JJ|NN.*>+}
#     PP: {<IN|TO|RP><NP|VB.*>} 
#     VP: {<VB.*|RB.*>+<PP|NP>*}
#     Sentence: {<UH>*<NP>+<MD>*<VP|PP|NP>+}
#     Sentence: {<PP|VP|NP>+}
#     Question: {<MD|WDT|DP|WRB|><MD>*<Sentence|NP|PP|VP>}
# """

### Section 2: Sentencizer transcripts

In [6]:
# Loop to sentencizer each sentences in the transcript_ls
text_result_dic = {}
for i, transcripts in enumerate(transcripts_ls):
    full_result = nlp_sentencizer(transcripts, grammars, nlp)
    text_result = list(map(lambda x: x[1], full_result))
    text_result_dic[i] = text_result
    print("Current transcript id: ", i)
    print("full result: ", full_result)
    print("\n")
    print("only text result: ", text_result)
    print("\n")

Current transcript id:  0
full result:  [['Sentence', 'ya hello good afternoon speak to nanny seah'], ['Sentence', 'please afternoon miss nanny my name is jaguar shao and'], ['Sentence', 'i m actually calling from insurance company miss nanny good time to speak for a while this is regarding our partnership charter_plus members and'], ['Sentence', 'as a result of that insurance company actually formulated'], ['Sentence', 'a very special thirtieth anniversay insurance bundle right called the i thirty \n']]


only text result:  ['ya hello good afternoon speak to nanny seah', 'please afternoon miss nanny my name is jaguar shao and', 'i m actually calling from insurance company miss nanny good time to speak for a while this is regarding our partnership charter_plus members and', 'as a result of that insurance company actually formulated', 'a very special thirtieth anniversay insurance bundle right called the i thirty \n']


Current transcript id:  1
full result:  [['Sentence', 'hello good a

### Section 3: Mapping sentence to each category

In [7]:
# Looping to store mapping result to dic
mapping_result_ls = []
for i, text in text_result_dic.items():
    category_ls = nlp_aspect_matching(text, nlp, 'greeting', False)
    print(category_ls)
    print("\n")
    mapping_result_ls.append(category_ls)

[['ask_for_permission', 'ya hello good afternoon speak to nanny seah'], ['purpose_of_call', 'please afternoon miss nanny my name is jaguar shao and'], ['purpose_of_call', 'i m actually calling from insurance company miss nanny good time to speak for a while this is regarding our partnership charter_plus members and'], ['purpose_of_call', 'as a result of that insurance company actually formulated'], ['purpose_of_call', 'a very special thirtieth anniversay insurance bundle right called the i thirty \n']]


[['opening', 'hello good afternoon just speak to miss leon michael from income ntuc'], ['ask_for_permission', 'free for one_or_two minutes if you are not busy okay'], ['purpose_of_call', 'calling behalf of your adviser xiao guo okay'], ['purpose_of_call', 'because we having this anniversary plan for the family i just check again you are single_or_married \n']]




### Section 4: Calculate sentiment score for each part

In [8]:
for mapping_result in mapping_result_ls:
    sentiment_ls = nlp_sentiment(mapping_result, tokenizer, sentiment_model)
    print(sentiment_ls)
    print("\n")

[['ask_for_permission', 'ya hello good afternoon speak to nanny seah', 'positive'], ['purpose_of_call', 'please afternoon miss nanny my name is jaguar shao and', 'neutral'], ['purpose_of_call', 'i m actually calling from insurance company miss nanny good time to speak for a while this is regarding our partnership charter_plus members and', 'neutral'], ['purpose_of_call', 'as a result of that insurance company actually formulated', 'neutral'], ['purpose_of_call', 'a very special thirtieth anniversay insurance bundle right called the i thirty \n', 'positive']]


[['opening', 'hello good afternoon just speak to miss leon michael from income ntuc', 'positive'], ['ask_for_permission', 'free for one_or_two minutes if you are not busy okay', 'neutral'], ['purpose_of_call', 'calling behalf of your adviser xiao guo okay', 'neutral'], ['purpose_of_call', 'because we having this anniversary plan for the family i just check again you are single_or_married \n', 'neutral']]


